<a href="https://colab.research.google.com/github/Elma-dev/CBOW_From_Scratch/blob/main/CBOW_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
corpus = [
    "The quick brown fox jumps over the lazy dog.",
    "She sells seashells by the seashore.",
    "How much wood would a woodchuck chuck if a woodchuck could chuck wood?",
    "To be or not to be, that is the question.",
    "It was the best of times, it was the worst of times.",
    "All happy families are alike; each unhappy family is unhappy in its own way.",
    "In the beginning, God created the heavens and the earth.",
    "Call me Ishmael.",
    "I have a dream that one day this nation will rise up and live out the true meaning of its creed.",
    "The only thing we have to fear is fear itself."
]

In [4]:
def create_data_from_corpus(corpus,cont_size=2):
  data={"features":[],"target":[]}
  context_size=cont_size
  for word in corpus:
    word=word.split(" ")
    sizeWindow=context_size*2+1
    for i in range(len(word)-sizeWindow+1):
      targetIdx=sizeWindow//2
      data["features"].append(word[i:i+targetIdx]+word[i+targetIdx+1:i+sizeWindow])
      data["target"].append(word[i+targetIdx])
      #print(word[i:i+targetIdx]+word[i+targetIdx+1:i+sizeWindow],word[i+targetIdx])
  return pd.DataFrame(data)

In [5]:
data=create_data_from_corpus(corpus)

In [6]:
data.head()

,features,target
0,"[The, quick, fox, jumps]",brown
1,"[quick, brown, jumps, over]",fox
2,"[brown, fox, over, the]",jumps
3,"[fox, jumps, the, lazy]",over
4,"[jumps, over, lazy, dog.]",the


In [7]:
text=" ".join(corpus)
text=list(set(text.split(" ")))

In [8]:
"""for w in text:
  data[w]=0
for i,tokens in enumerate(data.features):
  for t in tokens:
    data.at[i,t]=1"""

'for w in text:\n  data[w]=0\nfor i,tokens in enumerate(data.features):\n  for t in tokens:\n    data.at[i,t]=1'

In [9]:
data.head()

,features,target
0,"[The, quick, fox, jumps]",brown
1,"[quick, brown, jumps, over]",fox
2,"[brown, fox, over, the]",jumps
3,"[fox, jumps, the, lazy]",over
4,"[jumps, over, lazy, dog.]",the


In [10]:
def encoder(text):
  e={}
  for i,w in enumerate(text):
    c=np.zeros((len(text),))
    c[i]=1
    e[w]=c
  return pd.DataFrame(e)

In [11]:
dataEncoded=encoder(text)
dataEncoded.head()

,times.,wood?,and,of,I,best,fox,will,created,to,...,we,All,this,over,lazy,day,families,creed.,it,dog.
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

# build CBOW architecture
cbow = Sequential()
cbow.add(Embedding(input_dim=4*len(text), output_dim=100, input_length=332))
cbow.add(Lambda(lambda x: K.mean(x,axis=1), output_shape=(100,)))
cbow.add(Dense(len(text), activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
print(cbow.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 332, 100)          33200     
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 83)                8383      
                                                                 
Total params: 41,583
Trainable params: 41,583
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
X=[]
y=[]
for i,j in zip(data.features,data.target):
  l=[]
  for r in i:
    l.append(dataEncoded[r].values)
  X.append(l)
  y.append(dataEncoded[j].values)

In [15]:
cbow.fit(np.array(X).reshape(69,-1),np.array(y),epochs=100)

Epoch 1/100
3/3 [==============================] - 1s 12ms/step - loss: 4.4140
Epoch 2/100
3/3 [==============================] - 0s 8ms/step - loss: 4.4058
Epoch 3/100
3/3 [==============================] - 0s 7ms/step - loss: 4.4000
Epoch 4/100
3/3 [==============================] - 0s 13ms/step - loss: 4.3936
Epoch 5/100
3/3 [==============================] - 0s 7ms/step - loss: 4.3878
Epoch 6/100
3/3 [==============================] - 0s 9ms/step - loss: 4.3818
Epoch 7/100
3/3 [==============================] - 0s 8ms/step - loss: 4.3754
Epoch 8/100
3/3 [==============================] - 0s 8ms/step - loss: 4.3691
Epoch 9/100
3/3 [==============================] - 0s 8ms/step - loss: 4.3629
Epoch 10/100
3/3 [==============================] - 0s 7ms/step - loss: 4.3558
Epoch 11/100
3/3 [==============================] - 0s 7ms/step - loss: 4.3488
Epoch 12/100
3/3 [==============================] - 0s 7ms/step - loss: 4.3417
Epoch 13/100
3/3 [==============================] - 0s 7ms/

In [16]:
X=np.array(X).reshape(69,-1)
y=np.array(y).shape

In [17]:
np.array([X[0]])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [18]:
#Test
predict=cbow.predict(np.array([X[0]]))

1/1 [==============================] - 0s 100ms/step


In [19]:
dec=predict==predict.max()

In [23]:
dec=dec.astype(int)

In [24]:
np.argmax(predict)

38

In [26]:
for i in text:
  if dataEncoded[i][np.argmax(predict)]==1:
    print(i)

the
